In [37]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import re

In [10]:
df = pd.read_excel('auto/data/1_中國大陸_進出口總值(美元).xlsx', index_col = 0)
# df = df[df['category_x'] == '進口總值(含復進口)']
indexs = df['date'].tolist() 
label_im = ['進口總值(含復進口)']
# data_im_indexs = df.iloc[:, 1].tolist()  # Convert to list
data_im_values = df['import_美元'].tolist()  # Convert to list

# df = pd.read_excel('auto/data/1_中國大陸_進出口總值(美元).xlsx', index_col = 0)
# df = df[df['category_y'] == '出口總值(含復出口)']
label_ex = ['出口總值(含復出口)']
# data_ex_indexs = df.iloc[:, 1].tolist()  # Convert to list
data_ex_values = df['export_美元'].tolist()  # Convert to list

In [28]:
df['年'] = df['date'].apply(lambda x: x.split('年')[0])
df['月'] = df['date'].apply(lambda x: x.split('年')[1])

In [32]:
def convert_roc_to_gregorian(roc_date):
    # Split year and month
    year_str, month_str = roc_date.split('年')
    year = int(year_str)
    month = int(month_str.replace('月', ''))
    
    # Convert ROC year to Gregorian year
    gregorian_year = year + 1911
    
    # Return as formatted string
    return f"{gregorian_year}-{month:02d}"

# Apply the conversion to the dataframe
df['gregorian_date'] = df['date'].apply(convert_roc_to_gregorian)

In [33]:
df.sort_values('gregorian_date')

,category_x,date,中文貨名_x,國家_x,import_美元,category_y,中文貨名_y,國家_y,export_美元,年,月,gregorian_date
160,NaN,92年3月,NaN,NaN,NaN,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,0,92,3月,2003-03
161,NaN,92年4月,NaN,NaN,NaN,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,0,92,4月,2003-04
162,進口總值(含復進口),92年6月,可互換之搪孔或拉孔工具，工具機用,中國大陸,0.0,NaN,NaN,NaN,NaN,92,6月,2003-06
163,進口總值(含復進口),92年7月,可互換之搪孔或拉孔工具，工具機用,中國大陸,0.0,NaN,NaN,NaN,NaN,92,7月,2003-07
164,進口總值(含復進口),92年9月,可互換之搪孔或拉孔工具，工具機用,中國大陸,0.0,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,1,92,9月,2003-09
...,...,...,...,...,...,...,...,...,...,...,...,...
155,進口總值(含復進口),112年9月,可互換之搪孔或拉孔工具，工具機用,中國大陸,8.0,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,553,112,9月,2023-09
144,進口總值(含復進口),112年10月,可互換之搪孔或拉孔工具，工具機用,中國大陸,3.0,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,552,112,10月,2023-10
145,進口總值(含復進口),112年11月,可互換之搪孔或拉孔工具，工具機用,中國大陸,20.0,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,882,112,11月,2023-11
146,進口總值(含復進口),112年12月,可互換之搪孔或拉孔工具，工具機用,中國大陸,5.0,出口總值(含復出口),可互換之搪孔或拉孔工具，工具機用,中國大陸,868,112,12月,2023-12


In [19]:
data_im_values[2] == float('nan')

False

In [20]:
np.isnan(data_im_values[2])

True

In [18]:
type(data_im_values[2])

float

In [4]:
data_ex_values

[]

In [3]:
data_im_values

['0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '4',
 '3',
 '12',
 '20',
 '2',
 '0',
 '3',
 '5',
 '3',
 '4',
 '1',
 '1',
 '1',
 '3',
 '13',
 '3',
 '7',
 '4',
 '1',
 '1',
 '1',
 '17',
 '19',
 '26',
 '29',
 '5',
 '7',
 '7',
 '9',
 '2',
 '18',
 '26',
 '5',
 '11',
 '18',
 '20',
 '17',
 '20',
 '13',
 '23',
 '11',
 '6',
 '8',
 '8',
 '0',
 '1',
 '8',
 '14',
 '13',
 '18',
 '2',
 '28',
 '52',
 '23',
 '4',
 '19',
 '21',
 '8',
 '15',
 '2',
 '11',
 '24',
 '3',
 '2',
 '22',
 '3',
 '37',
 '21',
 '9',
 '23',
 '38',
 '47',
 '172',
 '43',
 '17',
 '8',
 '9',
 '6',
 '5',
 '4',
 '7',
 '8',
 '2',
 '3',
 '4',
 '4',
 '2',
 '14',
 '12',
 '11',
 '7',
 '6',
 '5',
 '11',
 '5',
 '12',
 '9',
 '11',
 '3',
 '8',
 '0',
 '7',
 '0',
 '8',
 '11',
 '0',
 '6',
 '1',
 '0',
 '0',
 '15',
 '5',
 '3',
 '4',
 '2',
 '9',
 '0',
 '0',
 '12',
 '20',
 '2',
 '5',
 '5',
 '13',
 '0',
 '1',
 '11',
 '6',
 '0',
 '11',
 '5',
 '2',
 '0',
 '0',
 '3',
 '6',
 '110',
 '2',
 '10',
 '5',
 '6',
 '11',
 '3',
 '27',
 '1',
 '23',
 '12',
 '27',
 '265',
 '6

In [2]:
'''各國產品搜索量'''
search_country = 'IN'
df = pd.read_excel(f'auto/data/2_google_{search_country}.xlsx', index_col = 0)
data_controller = df.iloc[:,0].tolist()
data_machine = df.iloc[:,1].tolist()

In [9]:
df.index.to_list()

[Timestamp('2012-01-01 00:00:00'),
 Timestamp('2012-01-02 00:00:00'),
 Timestamp('2012-01-03 00:00:00'),
 Timestamp('2012-01-04 00:00:00'),
 Timestamp('2012-01-05 00:00:00'),
 Timestamp('2012-01-06 00:00:00'),
 Timestamp('2012-01-07 00:00:00'),
 Timestamp('2012-01-08 00:00:00'),
 Timestamp('2012-01-09 00:00:00'),
 Timestamp('2012-01-10 00:00:00'),
 Timestamp('2012-01-11 00:00:00'),
 Timestamp('2012-01-12 00:00:00'),
 Timestamp('2012-01-13 00:00:00'),
 Timestamp('2012-01-14 00:00:00'),
 Timestamp('2012-01-15 00:00:00'),
 Timestamp('2012-01-16 00:00:00'),
 Timestamp('2012-01-17 00:00:00'),
 Timestamp('2012-01-18 00:00:00'),
 Timestamp('2012-01-19 00:00:00'),
 Timestamp('2012-01-20 00:00:00'),
 Timestamp('2012-01-21 00:00:00'),
 Timestamp('2012-01-22 00:00:00'),
 Timestamp('2012-01-23 00:00:00'),
 Timestamp('2012-01-24 00:00:00'),
 Timestamp('2012-01-25 00:00:00'),
 Timestamp('2012-01-26 00:00:00'),
 Timestamp('2012-01-27 00:00:00'),
 Timestamp('2012-01-28 00:00:00'),
 Timestamp('2012-01-

In [10]:
df = pd.read_excel('auto/data/3_主要國家工業生產增加率.xlsx', index_col = 0) 

In [24]:
df.iloc[:,0].values.tolist

<function ndarray.tolist>

In [18]:
df['中華民國'].values

array([ 15.05,  10.66,  10.64,   4.73,   5.62,   1.85,   0.56,   0.95,
        -1.34,   0.53,  -6.62, -10.12, -16.03,   6.75,  -1.94,  -0.58,
        -0.1 ,  -3.07,   0.86,   3.9 ,   6.22,   5.56,   7.45,   4.99,
        21.22,  -8.18,   0.31,   1.76,   3.3 ,   5.68,   5.86,   2.77,
         2.62,   2.7 ,   1.99,   9.03,  -0.28,   9.35,   4.81,   6.81,
         5.32,   8.38,   6.76,   8.29,  11.29,  11.02,   9.43,   9.  ,
        11.05,   3.64,   7.87,   2.48,  -0.53,  -0.04,  -2.96,  -6.4 ,
        -5.64,  -6.53,  -6.91,  -6.45,  -7.53,  -6.46,  -3.32,  -4.8 ,
         1.07,   2.32,   1.5 ,   9.56,   8.67,   7.52,  15.56,  10.55,
         5.64,  16.16,   8.33,   1.19,   3.27,   5.74,   2.55,   5.78,
         5.33,   2.62,   1.86,   3.93,   9.44,  -5.44,   5.22,   8.15,
         8.01,   0.13,   4.75,   1.29,   1.34,   7.96,   1.89,  -1.98,
         0.59,  -1.66,  -7.56,   2.26,  -1.89,  -0.23,   4.87,   3.1 ,
         0.75,  -0.38,   2.89,   8.77,  -1.83,  23.48,  12.95,   5.43,
      

In [25]:
df = pd.read_excel('auto/data/3_主要國家工業生產增加率.xlsx', index_col = 0) 
label_3 = df.columns.to_list()
indexs_3 = df.index.to_list()
data = {}
for i in range(0,df.shape[1]):
    tmp = df.iloc[:,i].tolist()
    data[label_3[i]] = tmp

In [2]:
# 寫入到舊有的excel檔的不同sheet裡面
path_new = 'auto/new_data/5_經濟數據_united-states.xlsx'
path_update = 'auto/data/5_經濟數據_united-states.json'

df = pd.read_excel(path_new) 
index_names = df[df['總經種類'] == 'Overview']['指標名稱'].values
with pd.ExcelWriter(engine = 'openpyxl', path = path_update, mode = 'a', if_sheet_exists = 'overlay') as writer: # mode='a'現有檔案讀寫
    tmp_df = pd.read_excel(path_update, sheet_name = None) 
    # 存到指定的sheet
    for col in index_names:
        if col in tmp_df.keys():
            old_df = pd.read_excel(path_update, sheet_name = col, index_col = 0) # 舊資料
            tmp = df[(df['總經種類'] == 'Overview') & (df['指標名稱'] == col)] # 新資料
            old_df = pd.concat([old_df, tmp])
            old_df.to_excel(writer, sheet_name = col) 
        else:
            print(col)
            tmp = tmp_df['Sheet1'][(tmp_df['Sheet1']['總經種類'] == 'Overview') & (tmp_df['Sheet1']['指標名稱'] == col)]
            tmp.to_excel(writer, sheet_name = col) 

ValueError: Invalid extension for engine '<property object at 0x0000015FA3821860>': 'json'

In [34]:
# 寫入到舊有的excel檔的不同sheet裡面
path_new = 'auto/new_data/5_經濟數據_united-states.xlsx'
path_update = 'auto/data/5_經濟數據_united-states_overview.json'

df = pd.read_excel(path_new) 
index_names = df[df['總經種類'] == 'Overview']['指標名稱'].values

try:
    jsonFile = open(path_update,'r')
    f =  jsonFile.read()   # 要先使用 read 讀取檔案
    df_update = json.loads(f)
    jsonFile.close()
except:
    df_update = {}

for tmp in index_names:
    if tmp not in df_update.keys():
        df_update[tmp] = []
        df_update[f'{tmp}_date'] = []
        df_update[f'{tmp}_unit'] = []
    if len(df_update[f'{tmp}_date'])>0:
        if df_update[f'{tmp}_date'][-1] == df[df['指標名稱'] == tmp]['公告日期'].values[0]:
            break        
    df_update[tmp].append(df[df['指標名稱'] == tmp]['Last'].values[0])
    df_update[f'{tmp}_date'].append(df[df['指標名稱'] == tmp]['公告日期'].values[0])
    df_update[f'{tmp}_unit'].append(df[df['指標名稱'] == tmp]['單位'].values[0])

json_data = json.dumps(df_update)
with open(path_update, "w") as json_file:
    json_file.write(json_data)

In [35]:
jsonFile = open("auto/data/5_經濟數據_united-states_overview.json",'r')
f =  jsonFile.read()   # 要先使用 read 讀取檔案
df5 = json.loads(f)      # 再使用 loads
jsonFile.close()

In [283]:
res = requests.get('https://www.sse.net.cn/index/singleIndex?indexType=gcspi')
soup = BeautifulSoup(res.content, 'lxml')
# Extract the tables from the HTML content
tables = soup.find_all('table')

# Function to extract table data
def parse_table(table, datatype = 0):
    data = []
    rows = table.find_all('tr')

    # Get the headers
    headers = [header.get_text() for header in rows[0].find_all('td')]
    data.append(headers)

    # Get the rest of the rows
    count = 0
    for row in rows[1:]:
        cols = row.find_all('td')
        cols = [ele.get_text(strip=True) for ele in cols]
        if datatype == 1:
            new_cols = [cols[i] for i in [0,1,3]]
        else:
            new_cols = [cols[i] for i in [0,1]]
        data.append(new_cols)
        count+=1
    
    if datatype == 1:
        return pd.DataFrame(data[2:])
    else:
        return pd.DataFrame(data[1:])

def get_column(datatype, soup):
    text = soup.find_all('div', {'class':'title2'})[datatype].text.split()[0]
    date_pattern = r"\d{4}-\d{2}"
    date_match = re.search(date_pattern, text).group()

    if datatype == 0:
        chinese_content_pattern = r"[\u4e00-\u9fff]+"
        chinese_match = re.search(chinese_content_pattern, text).group()

        return chinese_match, date_match
    else:
        split_content = re.split(r"/", text)

        # Extracting Chinese content before and after '/'
        before_slash = re.findall(r"[\u4e00-\u9fff]+", split_content[0])[0] if len(split_content) > 0 else []
        after_slash = re.findall(r"[\u4e00-\u9fff]+", split_content[1])[0] if len(split_content) > 1 else []
        return before_slash, after_slash, date_match

# Extract data from both tables
df_comprehensive = parse_table(tables[0],0)  # Comprehensive Punctuality Index
df_trunk_routes = parse_table(tables[1],1)   # Global Trunk Routes Punctuality

chinese_match, date_match = get_column(datatype = 0, soup = soup)
df_comprehensive.columns = [date_match, chinese_match]

before_slash, after_slash, date_match = get_column(datatype = 1, soup = soup)
df_trunk_routes.columns = [date_match, before_slash, after_slash]


In [295]:
res = requests.get('https://www.sse.net.cn/index/singleIndex?indexType=gcspi')
soup = BeautifulSoup(res.content, 'lxml')
# Extract the tables from the HTML content
tables = soup.find_all('table')

# Function to extract table data
def parse_table(table, datatype = 0):
    data = []
    rows = table.find_all('tr')

    # Get the headers
    headers = [header.get_text() for header in rows[0].find_all('td')]
    data.append(headers)

    # Get the rest of the rows
    count = 0
    for row in rows[1:]:
        cols = row.find_all('td')
        cols = [ele.get_text(strip=True) for ele in cols]
        if datatype == 1:
            new_cols = [cols[i] for i in [0,1,3]]
        else:
            new_cols = [cols[i] for i in [0,1]]
        data.append(new_cols)
        count+=1
    
    if datatype == 1:
        return pd.DataFrame(data[2:])
    else:
        return pd.DataFrame(data[1:])

def get_column(datatype, soup):
    text = soup.find_all('div', {'class':'title2'})[datatype].text.split()[0]
    date_pattern = r"\d{4}-\d{2}"
    date_match = re.search(date_pattern, text).group()

    if datatype == 0:
        chinese_content_pattern = r"[\u4e00-\u9fff]+"
        chinese_match = re.search(chinese_content_pattern, text).group()

        return chinese_match, date_match
    else:
        split_content = re.split(r"/", text)

        # Extracting Chinese content before and after '/'
        before_slash = re.findall(r"[\u4e00-\u9fff]+", split_content[0])[0] if len(split_content) > 0 else []
        after_slash = re.findall(r"[\u4e00-\u9fff]+", split_content[1])[0] if len(split_content) > 1 else []
        return before_slash, after_slash, date_match

# Extract data from both tables
df_comprehensive = parse_table(tables[0],0)  # Comprehensive Punctuality Index
df_trunk_routes = parse_table(tables[1],1)   # Global Trunk Routes Punctuality

chinese_match, date_match = get_column(datatype = 0, soup = soup)
df_comprehensive.columns = ['名稱', chinese_match]
df_comprehensive[chinese_match+'_time'] = date_match

before_slash, after_slash, date_match = get_column(datatype = 1, soup = soup)
df_trunk_routes.columns = ['名稱', before_slash, after_slash]
df_trunk_routes[f'{before_slash}_time'] = date_match
df_trunk_routes[f'{after_slash}_time'] = date_match

In [167]:
filepath = 'auto/new_data/'
df1 = pd.read_excel(f'{filepath}7_上海航運交易所_全球主幹航線綜合準班率指數.xlsx')
df2 = pd.read_excel(f'{filepath}7_上海航運交易所_全球主幹航線到離港與收發獲準班率指數.xlsx')
df2_1 = df2.iloc[:,[0,1,3]]
df2_2 = df2.iloc[:,[0,2,4]]

df7_2 = pd.read_excel('auto/new_data/7_上海航運交易所_港口班輪準確率.xlsx').astype(str)
df7_2_1 = df7_2.iloc[:,[0,1,7]]
df7_2_2 = df7_2.iloc[:,[0,2,8]]
df7_2_3 = df7_2.iloc[:,[0,3,9]]
df7_2_4 = df7_2.iloc[:,[0,4,10]]
df7_2_5 = df7_2.iloc[:,[0,5,11]]

df7_3 = pd.read_excel('auto/new_data/7_上海航運交易所_一帶一路航貿指數.xlsx', index_col = 0).astype(str)


jsonFile = open('auto/data/7_上海航運交易所_航運數據.json','r')
f =  jsonFile.read()   # 要先使用 read 讀取檔案
shipping_db = json.loads(f)      # 再使用 loads
shipping_db = {}
jsonFile.close()

df1_keys = list(df1['指數名稱'].values+'_time')+list(df1['指數名稱'].values)
df2_keys = list(df2['航線'].values+'_time')+list(df2['航線'].values)
df7_2_keys = list(df7_2['港口'].values+'_time')+list(df7_2['港口'].values)
df7_3_keys = list(df7_3['指數名稱'].values+'_time')+list(df7_3['指數名稱'].values)



def update_db(update_keys, update_data, shipping_db, data_type):
    if data_type == '7_1':
        query = '指數名稱'
        column = '全球主幹航線綜合準班率指數'
    elif data_type == '7_1_1':
        query = '航線'
        column = '全球主幹航線到離港準班率指數'
    elif data_type == '7_1_2':
        query = '航線'
        column = '全球主幹航線收發貨準班率指數'
    elif data_type == '7_2_1':
        query = '港口'
        column = '港口班輪準確率:準班率'
    elif data_type == '7_2_2':
        query = '港口'
        column = '港口班輪準確率:掛靠數'
    elif data_type == '7_2_3':
        query = '港口'
        column = '港口班輪準確率:班期综合服务水平'
    elif data_type == '7_2_4':
        query = '港口'
        column = '港口班輪準確率:在港时间(天)'
    elif data_type == '7_2_5':
        query = '港口'
        column = '港口班輪準確率:在泊時間(天)'
    elif data_type == '7_3':
        query = '指數名稱'
        column = '一帶一路航貿指數'   

    # 創立
    if column not in shipping_db.keys():
        shipping_db[f'{column}'] = {}


    shipping_keys = update_keys
    for tmp in shipping_keys:
        # 創立
        if tmp not in shipping_db[f'{column}'].keys():
            shipping_db[f'{column}'][tmp] = []
        
        # 防止重複增加
        tmp_new = tmp.split('_')[0]
        
        new_time = update_data[update_data[f'{query}'] == tmp_new].iloc[:,2].values[0]
        if '_time' in tmp:
            if len(shipping_db[f'{column}'][tmp_new+'_time']) > 0:
                if shipping_db[f'{column}'][tmp_new+'_time'][-1] == new_time:
                    break

        if '_time' in tmp:
            shipping_db[f'{column}'][tmp_new+'_time'].append(update_data[update_data[f'{query}'] == tmp_new].iloc[:,2].values[0])
        else:
            shipping_db[f'{column}'][tmp_new].append(update_data[update_data[f'{query}'] == tmp_new].iloc[:,1].values[0])
    print(f'航運資料庫更新完成{query}:{column}')
    return shipping_db

shipping_db = update_db(update_keys = df1_keys, update_data = df1, shipping_db = shipping_db, data_type = '7_1')
shipping_db = update_db(update_keys = df2_keys, update_data = df2_1, shipping_db = shipping_db, data_type = '7_1_1')
shipping_db = update_db(update_keys = df2_keys, update_data = df2_2, shipping_db = shipping_db, data_type = '7_1_2')
shipping_db = update_db(update_keys = df7_2_keys, update_data = df7_2_1, shipping_db = shipping_db, data_type = '7_2_1')
shipping_db = update_db(update_keys = df7_2_keys, update_data = df7_2_2, shipping_db = shipping_db, data_type = '7_2_2')
shipping_db = update_db(update_keys = df7_2_keys, update_data = df7_2_3, shipping_db = shipping_db, data_type = '7_2_3')
shipping_db = update_db(update_keys = df7_2_keys, update_data = df7_2_4, shipping_db = shipping_db, data_type = '7_2_4')
shipping_db = update_db(update_keys = df7_2_keys, update_data = df7_2_5, shipping_db = shipping_db, data_type = '7_2_5')
shipping_db = update_db(update_keys = df7_3_keys, update_data = df7_3, shipping_db = shipping_db, data_type = '7_3')

path_update = 'auto/data/7_上海航運交易所_航運數據.json'
json_data = json.dumps(shipping_db)
with open(path_update, "w") as json_file:
    json_file.write(json_data)

航運資料庫更新完成指數名稱:全球主幹航線綜合準班率指數
航運資料庫更新完成航線:全球主幹航線到離港準班率指數
航運資料庫更新完成航線:全球主幹航線收發貨準班率指數
航運資料庫更新完成港口:港口班輪準確率:準班率
航運資料庫更新完成港口:港口班輪準確率:掛靠數
航運資料庫更新完成港口:港口班輪準確率:班期综合服务水平
航運資料庫更新完成港口:港口班輪準確率:在港时间(天)
航運資料庫更新完成港口:港口班輪準確率:在泊時間(天)
航運資料庫更新完成指數名稱:一帶一路航貿指數


In [177]:
res = requests.get('https://www.sse.net.cn/index/singleIndex?indexType=brtvi')

soup = BeautifulSoup(res.content, 'lxml')

tables = soup.find_all('table')
rows = tables[0].find_all('tr')

data = []
for i in range(1,len(rows)):
    data.append([header.get_text() for header in rows[i].find_all('td')][0:2])

df = pd.DataFrame(data, columns = ['指數名稱', '指數'])
df['指數_time'] = soup.find_all('div', {'class':'title2'})[0].text.split()[1]
# df7_4 = pd.read_excel('auto/new_data/7_上海航運交易所_一帶一路貿易額指數.xlsx', index_col = 0).astype(str)

In [178]:
df

,指數名稱,指數,指數_time
0,综合指数,200.67,2024-09-25
1,蒙俄,407.59,2024-09-25
2,中亚,288.19,2024-09-25
3,南亚,180.54,2024-09-25
4,东南亚,199.50,2024-09-25
5,西亚,161.97,2024-09-25
6,北非,137.34,2024-09-25
7,欧洲,237.88,2024-09-25
8,大洋洲,159.73,2024-09-25


In [170]:
df7_4

,本期,与上期比涨跌(%)
指数/指标,,
综合指数,200.67,2.6
蒙俄,407.59,8.8
中亚,288.19,1.1
南亚,180.54,4.0
东南亚,199.5,0.7
西亚,161.97,7.6
北非,137.34,23.0
欧洲,237.88,1.3
大洋洲,159.73,-6.8
